In [29]:
import numpy as np
import scipy.stats as st
import matplotlib.pyplot as plt
import os, sys, pickle, time, re, csv
from itertools import groupby


In [5]:
###Folders

Input_folder = 'C:/Users/moham/team_mouse Dropbox/Mohamady El-Gaby/Taskspace_abstraction/Data/Intermediate_objects/'
Output_folder = 'C:/Users/moham/team_mouse Dropbox/Mohamady El-Gaby/Taskspace_abstraction/Results/Behaviour/'


'Download data from:  https://doi.org/10.17605/OSF.IO/3D9R2'

'Download data from:  https://doi.org/10.17605/OSF.IO/3D9R2'

In [6]:
from collections import defaultdict
def rec_dd():
    return defaultdict(rec_dd)
def indicesX2(z,thr):
    indicesx=[]
    for xx in range(0,len(z)):
        diffx=(z[xx,1]-z[xx,0]+1)
        if diffx>thr:
            indicesx.append(np.linspace\
                            (z[xx,0],z[xx,1],diffx))
    indices=np.asarray(indicesx)
    return(indices)

def unique_adjacent(a):
    return(np.asarray([k for k,g in groupby(a)]))


def remove_empty(xx):

    yy= [x for x in xx if len(x) > 0]
    return(yy)

def remove_nan(x):
    x=x[~np.isnan(x)]
    return(x)


In [8]:
###shortest distances 
from scipy.spatial import distance_matrix
from itertools import product
x=(0,1,2)
Task_grid=np.asarray(list(product(x, x)))

mapping_pyth={2:2,5:3,8:4}

distance_mat_raw=distance_matrix(Task_grid, Task_grid)
len_matrix=len(distance_mat_raw)
distance_mat=np.zeros((len_matrix,len_matrix))
for ii in range(len_matrix):
    for jj in range(len_matrix):
        if (distance_mat_raw[ii,jj]).is_integer()==False:
            hyp=int((distance_mat_raw[ii,jj])**2)
            distance_mat[ii,jj]=mapping_pyth[hyp]
        else:
            distance_mat[ii,jj]=distance_mat_raw[ii,jj]
mindistance_mat=distance_mat.astype(int)


In [92]:
##Scoring behaviour - example
mouse_recday='me11_02122021'
Tasks=np.load(Input_folder+'Task_data_'+mouse_recday+'.npy',allow_pickle=True)
ses_ind=0


Trial_times=np.load(Input_folder+'trialtimes_'+mouse_recday+'_'+str(ses_ind)+'.npy')//25
Location_raw=np.load(Input_folder+'Location_raw_'+mouse_recday+'_'+str(ses_ind)+'.npy')
Task=Tasks[ses_ind]

num_states=int(len(Trial_times[0])-1)
nodes=np.copy(Location_raw)
nodes[nodes>9]=np.nan

min_dists_task=np.asarray([mindistance_mat[Task[state_ind]-1,np.roll(Task,-1)[state_ind]-1]\
                           for state_ind in np.arange(num_states)])

length_matrix=np.zeros((len(Trial_times),num_states))
length_matrix[:]=np.nan
for trial_ind in np.arange(len(Trial_times)):
    for state_ind in np.arange(num_states):
        nodes_=unique_adjacent(remove_nan(nodes\
                    [Trial_times[trial_ind,state_ind]:Trial_times[trial_ind,state_ind+1]]))
        if nodes_[-1]==np.roll(Task,-1)[state_ind]:
            nodes_used_=nodes_[:-1]
        else:
            nodes_used_=nodes_
        length=len(nodes_used_)
        length_matrix[trial_ind,state_ind]=length
        
relative_distance_mat=length_matrix/min_dists_task ##distance relative to minimum distance
relative_distance_mat[relative_distance_mat<1]=np.nan

scores_matrix=relative_distance_mat==1 ##perfect transitions


In [99]:
print(relative_distance_mat)
print(scores_matrix)

[[1.         5.         1.66666667 1.        ]
 [2.33333333 3.         1.         3.        ]
 [1.66666667 4.         1.66666667 3.        ]
 [1.         3.         1.         3.        ]
 [1.         1.         1.66666667 4.        ]
 [1.66666667 1.         1.66666667 4.        ]
 [1.         1.         1.66666667 1.        ]
 [1.66666667 1.         1.66666667 4.        ]
 [1.66666667 1.         1.66666667 4.        ]
 [1.         3.         1.66666667 4.        ]
 [1.         1.         1.66666667 3.        ]
 [1.66666667 1.         1.66666667 1.        ]
 [1.         1.         1.66666667 4.        ]
 [1.66666667 1.         1.66666667 3.        ]
 [1.         1.         1.66666667 1.        ]
 [1.         1.         1.66666667 1.        ]
 [1.66666667 1.         1.66666667 1.        ]
 [1.         1.         3.         1.        ]
 [1.         3.         1.         1.        ]
 [1.         1.         1.66666667 1.        ]
 [1.         1.         1.66666667 1.        ]
 [1.         